In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# !wc -l fhvhv_tripdata_2021-01.csv

'wc' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [14]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [15]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [8]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
import pandas as pd

In [17]:
df_pandas = pd.read_parquet('fhvhv_tripdata_2021-01.parquet')

In [18]:
df_pandas = df_pandas.head(1001)

In [19]:
df_pandas.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [20]:
df_pandas = df_pandas[['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'shared_match_flag']]

# Rename the 'shared_match_flag' column to 'SR_Flag'
df_pandas.rename(columns={'shared_match_flag': 'SR_Flag'}, inplace=True)

df_pandas.dtypes

hvfhs_license_num               object
dispatching_base_num            object
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
SR_Flag                         object
dtype: object

In [21]:
df_pandas.shape

(1001, 7)

In [22]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', StringType(), True)])

Integer - 4 bytes
Long - 8 bytes

In [23]:
from pyspark.sql import types

In [28]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [29]:
# df = spark.read \
#     .option("header", "true") \
#     .schema(schema) \
#     .csv('fhvhv_tripdata_2021-01.csv')

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [30]:
df = df.repartition(24)

In [31]:
df.write.parquet('fhvhv/2021/01/')

In [32]:
df = spark.read.parquet('fhvhv/2021/01/')

In [34]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint, SR_Flag: string]

In [33]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [35]:
from pyspark.sql import functions as F

In [ ]:
F.

In [38]:
df\
    .withColumn('pickup_date', F.to_date(df.pickup_datetime))\
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))\
    .select('pickup_date','dropoff_date','PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-30|  2021-01-30|         256|         256|
| 2021-01-13|  2021-01-13|          61|          76|
| 2021-01-30|  2021-01-30|         113|         262|
| 2021-01-23|  2021-01-23|          22|         227|
| 2021-01-29|  2021-01-29|         198|          95|
| 2021-01-16|  2021-01-16|         235|          41|
| 2021-01-11|  2021-01-11|         138|         223|
| 2021-01-29|  2021-01-29|         155|          89|
| 2021-01-04|  2021-01-04|          26|         227|
| 2021-01-12|  2021-01-12|         221|         172|
| 2021-01-25|  2021-01-25|         258|         181|
| 2021-01-14|  2021-01-14|         140|          72|
| 2021-01-21|  2021-01-21|          49|         217|
| 2021-01-05|  2021-01-06|         169|         159|
| 2021-01-21|  2021-01-21|         235|          69|
| 2021-01-23|  2021-01-23|         146|       

In [39]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [40]:
crazy_stuff('B02884')

's/b44'

In [41]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [45]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02872|2021-01-30 01:41:29|2021-01-30 01:51:44|         256|         256|   NULL|
|           HV0005|              B02510|2021-01-13 01:39:57|2021-01-13 02:00:30|          61|          76|   NULL|
|           HV0003|              B02765|2021-01-30 02:25:19|2021-01-30 02:41:49|         113|         262|   NULL|
|           HV0005|              B02510|2021-01-23 03:08:47|2021-01-23 03:20:16|          22|         227|   NULL|
|           HV0003|              B02765|2021-01-29 21:13:39|2021-01-29 21:28:50|         198|          95|   NULL|
|           HV0003|              B02887|2021-01-16 17:11:49|2021-01-16 17:24:02|

In [43]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

Py4JJavaError: An error occurred while calling o177.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 69) (WIN-5VTIC6KQEVL executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun$mapP

In [48]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003').take(100)


[Row(pickup_datetime=datetime.datetime(2021, 1, 30, 1, 41, 29), dropoff_datetime=datetime.datetime(2021, 1, 30, 1, 51, 44), PULocationID=256, DOLocationID=256),
 Row(pickup_datetime=datetime.datetime(2021, 1, 30, 2, 25, 19), dropoff_datetime=datetime.datetime(2021, 1, 30, 2, 41, 49), PULocationID=113, DOLocationID=262),
 Row(pickup_datetime=datetime.datetime(2021, 1, 29, 21, 13, 39), dropoff_datetime=datetime.datetime(2021, 1, 29, 21, 28, 50), PULocationID=198, DOLocationID=95),
 Row(pickup_datetime=datetime.datetime(2021, 1, 16, 17, 11, 49), dropoff_datetime=datetime.datetime(2021, 1, 16, 17, 24, 2), PULocationID=235, DOLocationID=41),
 Row(pickup_datetime=datetime.datetime(2021, 1, 11, 20, 4, 13), dropoff_datetime=datetime.datetime(2021, 1, 11, 20, 11, 39), PULocationID=138, DOLocationID=223),
 Row(pickup_datetime=datetime.datetime(2021, 1, 29, 4, 54, 17), dropoff_datetime=datetime.datetime(2021, 1, 29, 5, 8, 9), PULocationID=155, DOLocationID=89),
 Row(pickup_datetime=datetime.datet

In [ ]:
!head -n 10 head.csv